# Creating, Planting, and Finding Signals Within Noise
Elizabeth Kapelevich

In [2]:
import math
import numpy as np
from scipy import interpolate
import matplotlib.pyplot as plt
import signals_and_noise as san

In [6]:
Time1, Position1 = np.loadtxt("mock_data_challenge_1.dat").T
Time2, Position2 = np.loadtxt("mock_data_challenge_2.dat").T

In [31]:
def fs_search(del_T_0, t_start, t_max, data_time_series, a, f, sigma, t_duration, del_T):
    
    frequency_list = []                                          # creating empty array to store other arrays
    sigma_list = []
    time_list = []
    crosscorr_list = []
    
    frequency_values = np.arange(0.01, 20, 1)                    # setting up ranges for which to run loops
    sigma_values = np.arange(0.01, 10, 1)

    for frequency in frequency_values:
        for s in sigma_values:
            times, C = san.cross_correlation(del_T_0, t_start, t_max,             # running calculation
                            data_time_series, a, frequency, s, t_duration, del_T)
            time_list.append(times)                              # appending arrays into empty lists
            crosscorr_list.append(C)
            frequency_list.append(frequency)
            sigma_list.append(s)

    frequency_list = frequency_list.flatten()                    # flattening 2-D arrays into 1-D
    sigma_list = sigma_list.flatten()
    time_list = time_list.flatten()
    crosscorr_list = crosscorr_list.flatten()
    
    largest_value_index = np.argmax(crosscorr_list)              # finding index of largest value in array
    frequency_correct = frequency_list[largest_value_index]      # finding corresponding value using index
    sigma_correct = sigma_list[largest_value_index]
    
    return frequency_correct, sigma_correct                      # returning two values

In [32]:
fs_search(del_T_0, t_start, t_max, data_time_series, a, f, sigma, t_duration, del_T)

ValueError: operands could not be broadcast together with shapes (2,2) (10000,) 

In [3]:
x = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])
y = np.array([1, 2, 3, 4, 5, 6, 7, 8, 9, 10])

In [4]:
def add(x, y):
    
    z = x + y
    
    return z

In [5]:
add(x, y)

array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20])